# Design and operations of energy technologies for a household (PV, heat pump, battery)


In this case study we find the optimal sizes and operations of a heat pump, a set of solar panels and a battery to power the energy needs (heat and electricity) of a household over 1 year. The household, which has certain heat and electricity demand profiles, has an existing connection to the grid and an existing gas boiler. The emissions of CO2 are taxed. We show a schematic representation of the system in the figure below.

<div>
<img src="figures/household_topology.png" width="500"/>
</div>

## Create templates
We set the input data path and in this directory we can add input data templates for the model configuration and the topology with the function create_optimization_templates.


In [ ]:
import adopt_net0 as adopt
import json
import pandas as pd
from pathlib import Path
import numpy as np

# Create folder for results
results_data_path = Path("./userData")
results_data_path.mkdir(parents=True, exist_ok=True)
# Create input data path and optimization templates
input_data_path = Path("./caseStudies/household")
input_data_path.mkdir(parents=True, exist_ok=True)
adopt.create_optimization_templates(input_data_path)

## Adapt Topology
We need to adapt the topology as well as the model configuration file to our case study. This can be done either in the file itself (Topology.json) or, as we do it here, via some lines of code.
For the topology, we need to change the following:
- Change nodes: household
- Change carriers: electricity, heat and gas
- Change investment periods: period1
- The options regarding the time frame we can leave at the default (one year with hourly operation)

In [ ]:
# Load json template
with open(input_data_path / "Topology.json", "r") as json_file:
    topology = json.load(json_file)
# Nodes
topology["nodes"] = ["household"]
# Carriers:
topology["carriers"] = ["electricity", "heat", "gas"]
# Investment periods:
topology["investment_periods"] = ["period1"]
# Save json template
with open(input_data_path / "Topology.json", "w") as json_file:
    json.dump(topology, json_file, indent=4)


## Adapt Model Configurations
Now, we need to adapt the model configurations respectively. In this case, we don't use any particular algorithm (clustering, time staging etc), so here we only specify what we want to minimize - costs in this example - and set the gap for the MILP solver to a desired value. 


In [ ]:
# Load json template
with open(input_data_path / "ConfigModel.json", "r") as json_file:
    configuration = json.load(json_file)
# Change objective
configuration["optimization"]["objective"]["value"] = "costs"
# Set MILP gap
configuration["solveroptions"]["mipgap"]["value"] = 0.02
# Save json template
with open(input_data_path / "ConfigModel.json", "w") as json_file:
    json.dump(configuration, json_file, indent=4)

## Define input data
We first create all required input data files based on the topology file and then add the existing technologies (gas boiler, 3kW capacity) and potential new technologies (heat pump, PV, battery). Since we have potentially a PV, we need to specify the location of the household.
Additionally, we copy over technology data to the input data folder


In [ ]:
adopt.create_input_data_folder_template(input_data_path)

# Add heat pump, PV and battery as new technologies
with open(input_data_path / "period1" / "node_data" / "household" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["Photovoltaic","HeatPump_AirSourced","Storage_Battery"]
technologies["existing"] = {"Boiler_Small_NG": 0.003}

with open(input_data_path / "period1" / "node_data" / "household" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)

# Copy over technology files
adopt.copy_technology_data(input_data_path)

# Specify location of the household
node_location = pd.read_csv(input_data_path / "NodeLocations.csv", sep=';', index_col=0, header=0)
node_location.at['household', 'lon'] = 4.9
node_location.at['household', 'lat'] = 52
node_location.at['household', 'alt'] = 10
node_location = node_location.reset_index()
node_location.to_csv(input_data_path / "NodeLocations.csv", sep=';', index=False)


## Read demand data, import limits/price
In this case study we want to model an hourly heat and electricity demand of the household, with variable electricity prices. In addition, we set a fixed value for gas and electricity import limits and gas prices. The examplary demand profiles and electricity profiles are provided with the package. As such we:
- Read day-ahead electricity prices (hourly for the Netherlands in 2023)
- Read hourly heat (total 7.3MWh) and electricity demand (total 4MWh)
- Limit import of gas (5kW) and electricity (8kW)
- Set an import price on gas of 120 EUR/MWh

Finally, we set a carbon tax to 100 EUR/t.

N.B all the data should be in MW, EUR/MWh, EUR/t, t/MWh

In [ ]:
# Read hourly data from Excel
household_hourly_data = adopt.load_household_data()

# Save the hourly data to the carrier's file in the case study folder
# electricity demand and price
el_price = household_hourly_data.iloc[:, 2]
el_demand = household_hourly_data.iloc[:, 1]
heat_demand = household_hourly_data.iloc[:, 0]
adopt.fill_carrier_data(input_data_path, value_or_data=el_price, columns=['Import price'], carriers=['electricity'], nodes=['household'])
adopt.fill_carrier_data(input_data_path, value_or_data=el_price, columns=['Export price'], carriers=['electricity'], nodes=['household'])
adopt.fill_carrier_data(input_data_path, value_or_data=el_demand, columns=['Demand'], carriers=['electricity'], nodes=['household'])
adopt.fill_carrier_data(input_data_path, value_or_data=heat_demand, columns=['Demand'], carriers=['heat'], nodes=['household'])

# Set import limits/cost
adopt.fill_carrier_data(input_data_path, value_or_data=0.005, columns=['Import limit'], carriers=['gas'], nodes=['household'])
adopt.fill_carrier_data(input_data_path, value_or_data=0.008, columns=['Import limit'], carriers=['electricity'], nodes=['household'])
adopt.fill_carrier_data(input_data_path, value_or_data=120, columns=['Import price'], carriers=['gas'], nodes=['household'])

# Set carbon emission price
carbon_price = np.ones(8760)*100
carbon_cost_path = "./caseStudies/household/period1/node_data/household/CarbonCost.csv"
carbon_cost_template = pd.read_csv(carbon_cost_path, sep=';', index_col=0, header=0)
carbon_cost_template['price'] = carbon_price
carbon_cost_template = carbon_cost_template.reset_index()
carbon_cost_template.to_csv(carbon_cost_path, sep=';', index=False)

## Run model
Now, we have defined all required data to run the model

In [ ]:
m = adopt.ModelHub()
m.read_data(input_data_path)
m.quick_solve()

## Visualization
The results can be inspected using the provided [visualization platform](https://resultvisualization.streamlit.app/) for some basic plots. The figures below are screenshots from the visualization platform. The results data are saved in the userData folder.

### Size of the new technologies
The optimization shows that the only technology installed is the heat pump. This means that, in these conditions, it is not economically favourable to install a PV or a battery. The size of the heat pump is 0.6 kW (based on electricity input, not heat output), which complements the existing gas boiler in covering the household's heat demand.

## Electricity and Heat requirements
<div>
<img src="figures/household_size.png" width="700"/>
</div>

### Heat pump operation
<div>
<img src="figures/household_operation.png" width="700"/>
</div>

